In [106]:
import sys
from copy import deepcopy
import torch
from torch import nn

# # Initialize Pygame
# pygame.init()

# Constants
WIDTH, HEIGHT = 600, 600
GRID_SIZE = 8
CELL_SIZE = WIDTH // GRID_SIZE
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 128, 0)

# # Create the game window
# screen = pygame.display.set_mode((WIDTH, HEIGHT))
# pygame.display.set_caption("6x6 Othello")

# Initialize the game board
board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
board[3][4] = board[4][3] = 2  # BLACK
board[3][3] = board[4][4] = 1  # White

# Current player (1 for White, 2 for Black)
current_player = 2

# def draw_board():
#     screen.fill(GREEN)
#     for x in range(GRID_SIZE):
#         for y in range(GRID_SIZE):
#             pygame.draw.rect(screen, BLACK, (x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)
#             if board[y][x] == 1:
#                 pygame.draw.circle(screen, WHITE, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)
#             elif board[y][x] == 2:
#                 pygame.draw.circle(screen, BLACK, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)

def is_valid_move(x, y,board):
    if board[y][x] != 0:
        return False
    
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board):
            return True
    return False

def check_direction(x, y, dx, dy,board):
    opponent = 1 if current_player == 2 else 2
    x, y = x + dx, y + dy
    if not (0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE) or board[y][x] != opponent:
        return False
    while 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE:
        if board[y][x] == 0:
            return False
        if board[y][x] == current_player:
            return True
        x, y = x + dx, y + dy
    return False

def make_move(x, y,board):
    board[y][x] = current_player
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board):
            flip_direction(x, y, dx, dy,board)

def flip_direction(x, y, dx, dy,board):
    x, y = x + dx, y + dy
    while board[y][x] != current_player:
        board[y][x] = current_player
        x, y = x + dx, y + dy

def switch_player():
    global current_player
    current_player = 1 if current_player == 2 else 2



In [107]:
data = []
with open("othello_dataset.csv","r") as f:
    flag = 1
    for line in f:       
        if flag:
            flag = 0
            continue
        data.append(line.strip().split(',')[1:])

In [108]:
import numpy

def FLATTEN(arr):
    ret = []
    for x in arr: ret = ret + x
    return ret

def makeData(data):
    inputs = []
    labels = []
    for y,moves in data:
        y = int(y)
        board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        board[3][4] = board[4][3] = 2  # White
        board[3][3] = board[4][4] = 1  # Black
        
       
        game_length = len(moves)//2
        discount = 0.99

        for i in range(0,len(moves),2):
            if current_player == 2:
                inputs.append(FLATTEN(board))
                labels.append([y * (0.99**(64- i//2 + 1))])
            grid_x = ord(moves[i]) - ord('a')
            grid_y = int(moves[i+1]) 
            
            make_move(grid_x, grid_y-1,board)
            switch_player()
        
    return inputs,labels
X,Y = makeData(data)

In [109]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32,16),
            nn.LeakyReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
 
device = "cuda"
model = NeuralNetwork().to(device)
learning_rate = 1e-3 * 5
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [110]:
X = torch.tensor(X,device=device,dtype=torch.float32)
Y = torch.tensor(Y,device=device,dtype=torch.float32)
epochs = 1000
for epoch in range(epochs):
    print(epoch)
    pred  = model(X)
    loss = loss_fn(pred,Y)
    print(torch.sum(loss))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


0
tensor(0.7446, device='cuda:0', grad_fn=<SumBackward0>)
1
tensor(0.7102, device='cuda:0', grad_fn=<SumBackward0>)
2
tensor(0.6837, device='cuda:0', grad_fn=<SumBackward0>)
3
tensor(0.6548, device='cuda:0', grad_fn=<SumBackward0>)
4
tensor(0.6252, device='cuda:0', grad_fn=<SumBackward0>)
5
tensor(0.5977, device='cuda:0', grad_fn=<SumBackward0>)
6
tensor(0.5747, device='cuda:0', grad_fn=<SumBackward0>)
7
tensor(0.5568, device='cuda:0', grad_fn=<SumBackward0>)
8
tensor(0.5434, device='cuda:0', grad_fn=<SumBackward0>)
9
tensor(0.5338, device='cuda:0', grad_fn=<SumBackward0>)
10
tensor(0.5268, device='cuda:0', grad_fn=<SumBackward0>)
11
tensor(0.5218, device='cuda:0', grad_fn=<SumBackward0>)
12
tensor(0.5181, device='cuda:0', grad_fn=<SumBackward0>)
13
tensor(0.5153, device='cuda:0', grad_fn=<SumBackward0>)
14
tensor(0.5132, device='cuda:0', grad_fn=<SumBackward0>)
15
tensor(0.5115, device='cuda:0', grad_fn=<SumBackward0>)
16
tensor(0.5103, device='cuda:0', grad_fn=<SumBackward0>)
17
tens

In [111]:
torch.save(model.state_dict(), 'model1.pth')